# 🗣️ Run Text-to-Speech App on Google Colab

Run this notebook to launch the app in the cloud for free.

In [ ]:
# 1. Install Dependencies
!pip install streamlit edge-tts

In [ ]:
# 2. Download the App Code (We create it here for convenience, or you can git clone)
app_code = """
import streamlit as st
import edge_tts
import asyncio
import tempfile
import os

# Set page configuration
st.set_page_config(
    page_title="Advanced Text-to-Speech",
    page_icon="🗣️",
    layout="wide"
)

# Custom title
st.markdown("""
<h1 style='text-align: center; color: #4CAF50;'>🗣️ Advanced Text-to-Speech Generator</h1>
<p style='text-align: center; font-size: 1.1em;'>Generate high-quality speech from text using Microsoft Edge TTS</p>
<hr>
""", unsafe_allow_html=True)

VOICES = {
    "🇺🇸 Male - Guy (Neural)": "en-US-GuyNeural",
    "🇺🇸 Female - Jenny (Neural)": "en-US-JennyNeural",
    "🇺🇸 Female - Aria (Neural)": "en-US-AriaNeural",
    "🇺🇸 Male - Brandon (Neural)": "en-US-BrandonNeural",
    "🇬🇧 Female - Sonia (Neural)": "en-GB-SoniaNeural",
    "🇬🇧 Male - Ryan (Neural)": "en-GB-RyanNeural",
}

def main():
    with st.sidebar:
        st.header("⚙️ Configuration")
        voice_label = st.selectbox("Select Voice", list(VOICES.keys()), index=1)
        voice_key = VOICES[voice_label]
        speed_val = st.slider("Rate (Speed)", -50, 100, 0, format="%d%%")
        rate_str = f"{'+' if speed_val >= 0 else ''}{speed_val}%"
        pitch_val = st.slider("Pitch", -50, 50, 0, format="%dHz")
        pitch_str = f"{'+' if pitch_val >= 0 else ''}{pitch_val}Hz"
        output_format = st.radio("Format", ["mp3", "wav"], horizontal=True)
    
    col1, col2 = st.columns([2, 1])
    with col1:
        st.subheader("📜 Input Text")
        text = st.text_area("Type or paste your text here...", height=350, placeholder="Hello! This is a test.")
        generate_btn = st.button("🔊 Generate Speech", type="primary", use_container_width=True)

    with col2:
        st.subheader("🎧 Output")
        if generate_btn:
            if not text.strip():
                st.warning("⚠️ Please enter some text first!")
            else:
                try:
                    with st.spinner("Generating audio..."):
                        with tempfile.NamedTemporaryFile(delete=False, suffix=f".{output_format}") as tmp_file:
                            tmp_path = tmp_file.name
                        
                        async def generate():
                            communicate = edge_tts.Communicate(text, voice_key, rate=rate_str, pitch=pitch_str)
                            await communicate.save(tmp_path)
                        
                        asyncio.run(generate())
                        st.success("✅ Generation Complete!")
                        st.audio(tmp_path, format=f"audio/{output_format}")
                        with open(tmp_path, "rb") as f:
                            st.download_button(f"⬇️ Download {output_format.upper()}", f, f"speech.{output_format}", f"audio/{output_format}", use_container_width=True)
                except Exception as e:
                    st.error(f"❌ Error: {str(e)}")

if __name__ == "__main__":
    main()
"""

with open("app.py", "w", encoding="utf-8") as f:
    f.write(app_code)


In [ ]:
# 3. Expose to Internet via LocalTunnel
!npm install localtunnel
!streamlit run app.py &>/content/logs.txt &
!npx localtunnel --port 8501